In [1]:
from rdflib import Graph
import numpy as np
import pandas as pd

In [2]:
# get all triples from ontology in easy format (without urls)
g = Graph()
g.parse("data/KG.ttl")
data = []
i=0
for s,p,o in g.triples((None,None,None)):
    if '#' in s:
        s = s.split("#")[1]
    if '#' in p:
        p = p.split("#")[1]
    if '#' in o:
        o = o.split("#")[1]
    data.append([s, p, o])

In [3]:
# get seen triples of format (drug, resultedIn, symptom) from data
drugcodes = []
symptoms = []
seen = []
for triple in data:
    if 'resultedIn' in triple[1]:
        seen.append(triple)
        if triple[0] not in drugcodes:
            drugcodes.append(triple[0])
        if triple[2] not in symptoms:
            symptoms.append(triple[2])

# create all possible combinations of drug-symptom 
links = []
for drug in drugcodes:
    for symptom in symptoms:
        links.append([drug, 'resultedIn', symptom])

X_unseen = np.array(links)

In [7]:
# get MLP probabilities from pickle files
df = pd.read_pickle("results/DistMult/DistMult_100.pkl") 
seen_probs = []
unseen_probs = []
for i,row in df.iterrows():
    s = row['statement'].split()
    p = row['prob']
    if s in seen:
        seen_probs.append((s,p))
    else:
        unseen_probs.append((s,p))
    
df_seen = pd.DataFrame(seen_probs, columns=['link', 'prob'])
df_unseen = pd.DataFrame(unseen_probs, columns=['link', 'prob'])
print(df_unseen)
print(df_seen)
